In [27]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
import os
import tarfile
from keras.optimizers import adam, rmsprop, adadelta
import csv
from keras.models import Sequential
from keras.layers import Dense,Dropout
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import preprocessing
from sklearn.tree import  DecisionTreeRegressor
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from keras import backend as K
from keras import optimizers

In [4]:
#导入数据
data=data = pd.read_csv("E:/ELEC3D_for_Chen/train_data.csv")

In [5]:
#查看
data.head()

,label,Upper_quartile,Lower_Quantile,quartile_deviation,Median,average_value,standard_deviation,Dispersion_coefficient,skewness,kurtosis,mean_width,surface,volume
0,0.077249,4.000000,15.000000,11.000000,9.666667,9.483417,5.795143,0.611082,0.007080,-1.222705,1.901743,11986.0,4000.0
1,0.075093,5.000000,14.333333,9.333333,10.000000,9.545917,5.722319,0.599452,-0.013571,-1.189178,1.910220,11968.0,4000.0
2,0.077239,4.333333,14.666667,10.333333,9.666667,9.522583,5.765345,0.605439,-0.004310,-1.212922,1.906275,11972.0,4000.0
3,0.077881,4.333333,14.666667,10.333333,9.666667,9.538583,5.770752,0.604990,-0.009660,-1.204863,1.903251,11996.0,4000.0
4,0.077418,5.000000,14.666667,9.666667,10.000000,9.545917,5.773821,0.604847,-0.006533,-1.208783,1.881173,12152.0,4000.0


In [6]:
#分割标签和特征项
y = data['label'].copy()
X = data[['Upper_quartile','Lower_Quantile','quartile_deviation','Median','average_value','standard_deviation',
                     'Dispersion_coefficient','skewness','kurtosis','mean_width','surface']].copy()

In [7]:
#分割训练集和验证集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
#查看是否缺失
X_train.isnull().any().describe()

count        11
unique        1
top       False
freq         11
dtype: object

In [9]:
#归一化处理
X=(X- X.min()) / (X.max() - X.min())
X_train = (X_train- X.min()) / (X.max() - X.min())
X_test=(X_test- X.min()) / (X.max() - X.min())

In [10]:
#定义rmse，mae，R_Squared
def compute_rmse(y1, y2):
    return np.mean((y1 - y2) ** 2) ** 0.5


def compute_mae(y1, y2):
    return np.sum(np.abs(y1-y2))/len(np.abs(y1-y2))


def compute_R_Squared(y1 , y2):
    return 1 - np.mean((y1 - y2) ** 2) / np.var(y_test)

In [13]:
#线性模型
LR =  LinearRegression()
LR.fit(X_train,y_train)
LR.predict(X_test)

array([0.07669263, 0.07644563, 0.07686769, ..., 0.0760339 , 0.07720664,
       0.07707427])

In [14]:
compute_rmse(LR.predict(X_test), y_test)

0.0007171715808704012

In [15]:
compute_mae(LR.predict(X_test), y_test)

0.0005750193091546919

In [16]:
compute_R_Squared(LR.predict(X_test), y_test)

0.36504689872509

In [17]:
#搭建回归树
TREE=DecisionTreeRegressor()
TREE.fit(X_train, y_train)
TREE.predict(X_test)

array([0.07731017, 0.07637092, 0.07676253, ..., 0.07665251, 0.07790401,
       0.07666189])

In [18]:
compute_rmse(TREE.predict(X_test), y_test)

0.0010154397884606643

In [19]:
compute_mae(TREE.predict(X_test), y_test)

0.0008126386003386375

In [20]:
compute_R_Squared(TREE.predict(X_test), y_test)

-0.27292805613662807

In [21]:
#搭建NN模型
#自定义标准rmse mae r_square
def rmse(y_true, y_pred):
    return K.mean((y_true - y_pred) ** 2) ** 0.5

def mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=-1)

def r_square(y_true, y_pred):
    SSR = K.mean(K.square(y_pred-K.mean(y_true)),axis=-1)
    SST = K.mean(K.square(y_true-K.mean(y_true)),axis=-1)
    return SSR/SST

In [38]:
model=Sequential()
model.add(Dense(units=50,input_dim=11,activation='relu'))
model.add(Dense(units=30,activation='relu'))
model.add(Dense(units=1))
adam=optimizers.Adam(lr=10, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='mse', optimizer=adam,metrics =[rmse,mae,r_square])
model.fit(X,y, batch_size=100, epochs=150,validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/150
8000/8000 [==============================] - 1s 70us/step - loss: 5164421962.7266 - rmse: 10991.9462 - mae: 10947.1519 - r_square: 7173495285662731264.0000 - val_loss: 2084.4906 - val_rmse: 45.6562 - val_mae: 45.6562 - val_r_square: 27623969603584.0000
Epoch 2/150
8000/8000 [==============================] - 0s 9us/step - loss: 2059.0653 - rmse: 45.3766 - mae: 45.3766 - r_square: 24978369209446.3984 - val_loss: 2028.3188 - val_rmse: 45.0369 - val_mae: 45.0369 - val_r_square: 26879570719539.1992
Epoch 3/150
8000/8000 [==============================] - 0s 9us/step - loss: 1992.8151 - rmse: 44.6403 - mae: 44.6403 - r_square: 504269844288307.1875 - val_loss: 1954.1104 - val_rmse: 44.2053 - val_mae: 44.2053 - val_r_square: 25896158951833.6016
Epoch 4/150
8000/8000 [==============================] - 0s 9us/step - loss: 1912.4648 - rmse: 43.7308 - mae: 43.7308 - r_square: 34692073658675.1992 - val_loss: 1868.0320 - val_rmse: 43.2207 

8000/8000 [==============================] - 0s 10us/step - loss: 18.3580 - rmse: 4.2807 - mae: 4.2807 - r_square: 2595260634332.7998 - val_loss: 15.7250 - val_rmse: 3.9655 - val_mae: 3.9655 - val_r_square: 208394902713.6000
Epoch 35/150
8000/8000 [==============================] - 0s 10us/step - loss: 13.5643 - rmse: 3.6793 - mae: 3.6793 - r_square: 666144267468.4000 - val_loss: 11.5370 - val_rmse: 3.3966 - val_mae: 3.3966 - val_r_square: 152894099760.0000
Epoch 36/150
8000/8000 [==============================] - 0s 9us/step - loss: 9.8884 - rmse: 3.1412 - mae: 3.1412 - r_square: 362713297217.4000 - val_loss: 8.3488 - val_rmse: 2.8894 - val_mae: 2.8894 - val_r_square: 110643092956.8000
Epoch 37/150
8000/8000 [==============================] - 0s 9us/step - loss: 7.1086 - rmse: 2.6630 - mae: 2.6630 - r_square: 1586127559420.5000 - val_loss: 5.9559 - val_rmse: 2.4405 - val_mae: 2.4405 - val_r_square: 78931409392.0000
Epoch 38/150
8000/8000 [==============================] - 0s 10us/step

8000/8000 [==============================] - 0s 9us/step - loss: 8.4843e-07 - rmse: 9.1921e-04 - mae: 7.3644e-04 - r_square: 139.3116 - val_loss: 7.9217e-07 - val_rmse: 8.8698e-04 - val_mae: 7.1133e-04 - val_r_square: 165.8706
Epoch 70/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4781e-07 - rmse: 9.1875e-04 - mae: 7.3609e-04 - r_square: 1275.4006 - val_loss: 7.9134e-07 - val_rmse: 8.8651e-04 - val_mae: 7.1076e-04 - val_r_square: 142.0531
Epoch 71/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4768e-07 - rmse: 9.1804e-04 - mae: 7.3602e-04 - r_square: 457.2648 - val_loss: 7.9100e-07 - val_rmse: 8.8631e-04 - val_mae: 7.1049e-04 - val_r_square: 131.0058
Epoch 72/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4762e-07 - rmse: 9.1888e-04 - mae: 7.3597e-04 - r_square: 201.3269 - val_loss: 7.9087e-07 - val_rmse: 8.8623e-04 - val_mae: 7.1038e-04 - val_r_square: 126.5685
Epoch 73/150
8000/8000 [============================

8000/8000 [==============================] - 0s 9us/step - loss: 8.4769e-07 - rmse: 9.1912e-04 - mae: 7.3599e-04 - r_square: 16.4379 - val_loss: 7.9038e-07 - val_rmse: 8.8594e-04 - val_mae: 7.0991e-04 - val_r_square: 107.0171
Epoch 104/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4777e-07 - rmse: 9.1879e-04 - mae: 7.3596e-04 - r_square: 47.0719 - val_loss: 7.9141e-07 - val_rmse: 8.8655e-04 - val_mae: 7.1081e-04 - val_r_square: 144.2336
Epoch 105/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4768e-07 - rmse: 9.1821e-04 - mae: 7.3607e-04 - r_square: 149.4508 - val_loss: 7.9058e-07 - val_rmse: 8.8606e-04 - val_mae: 7.1011e-04 - val_r_square: 115.5872
Epoch 106/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4786e-07 - rmse: 9.1857e-04 - mae: 7.3613e-04 - r_square: 32.2980 - val_loss: 7.9020e-07 - val_rmse: 8.8584e-04 - val_mae: 7.0971e-04 - val_r_square: 97.3994
Epoch 107/150
8000/8000 [=============================

8000/8000 [==============================] - 0s 9us/step - loss: 8.4857e-07 - rmse: 9.1979e-04 - mae: 7.3634e-04 - r_square: inf - val_loss: 7.9320e-07 - val_rmse: 8.8757e-04 - val_mae: 7.1196e-04 - val_r_square: 192.2377
Epoch 138/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4803e-07 - rmse: 9.1860e-04 - mae: 7.3605e-04 - r_square: 3106.0879 - val_loss: 7.9161e-07 - val_rmse: 8.8666e-04 - val_mae: 7.1095e-04 - val_r_square: 150.1233
Epoch 139/150
8000/8000 [==============================] - 0s 10us/step - loss: 8.4808e-07 - rmse: 9.1854e-04 - mae: 7.3607e-04 - r_square: 189.0904 - val_loss: 7.9248e-07 - val_rmse: 8.8716e-04 - val_mae: 7.1153e-04 - val_r_square: 174.0692
Epoch 140/150
8000/8000 [==============================] - 0s 9us/step - loss: 8.4897e-07 - rmse: 9.1956e-04 - mae: 7.3686e-04 - r_square: 156.1833 - val_loss: 7.9232e-07 - val_rmse: 8.8707e-04 - val_mae: 7.1143e-04 - val_r_square: 169.8786
Epoch 141/150
8000/8000 [============================